In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthBegin, MonthEnd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import ta
import pandas_ta

In [2]:
import fxcmpy
fxcm_token = 'YOUR_FXCM_TOCKEN'

In [10]:
startdate = '2010-01-01'
enddate = '2022-11-01'
symbol = 'SPX500'
data_dir = '/Users/dplavcan/Documents/ML_project/data/sp500_5min.pickle'

In [4]:
def load_data_from_FXCM(symbol, fxcm_token):
    api = fxcmpy.fxcmpy(access_token=fxcm_token)
    dates = pd.date_range(startdate, enddate, freq='M')
    data = []
    for date in dates:
        t1 = date - MonthBegin(1)
        t2 = date + MonthBegin(1)
        print(f'{t1} - {t2}')
        data.append(
            api.get_candles(symbol, start=t1, end=t2, period='m5').iloc[:-1]
        )
    df = pd.concat(data).asfreq('5min')
    return(df)

In [5]:
if os.path.exists(data_dir):
    print('loading data from saved file')
    df = pd.read_pickle(data_dir)
else:
    print('loading data from fxcm')
    df = load_data_from_FXCM(symbol, fxcm_token)
    df.to_pickle(data_dir)

loading data from saved file


In [6]:
df['rsi_14'] = ta.momentum.RSIIndicator(df['bidclose'], window=14).rsi()
df['rsi_2'] = ta.momentum.RSIIndicator(df['bidclose'], window=2).rsi()

In [7]:
supertrend = pandas_ta.supertrend(
    high=df['bidhigh'],
    low=df['bidlow'],
    close=df['bidclose'])

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [16]:
data_1 = '2022-10-24'
data_2 = '2022-10-28'

df_plot = df.loc[data_1:data_2]
supertrend_plot = supertrend.loc[data_1:data_2]


# Declare plotly figure (go)
fig=go.Figure()
# add subplot properties when initializing fig variable ***don't forget to import plotly!!!***
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.5,0.2, 0.2])


fig.add_trace(go.Candlestick(x=df_plot.index,
                open=df_plot['bidopen'],
                high=df_plot['bidhigh'],
                low=df_plot['bidlow'],
                close=df_plot['bidclose'],
                            name=symbol))

fig.add_trace(go.Scatter(x=supertrend_plot.index, 
                         y=supertrend_plot['SUPERTs_7_3.0'], 
                         opacity=0.7, 
                         line=dict(color='red', width=2), 
                         name='SUPERTs_7_3.0'))

fig.add_trace(go.Scatter(x=supertrend_plot.index, 
                         y=supertrend_plot['SUPERTl_7_3.0'], 
                         opacity=0.7, 
                         line=dict(color='green', width=2), 
                         name='SUPERTl_7_3.0'))

fig.add_trace(go.Scatter(x=df_plot.index, 
                     y=df_plot['rsi_2'],
                         name='rsi_2'
                    ), row=2, col=1)                   

fig.add_trace(go.Scatter(x=df_plot.index, 
                     y=df_plot['rsi_14'],
                         name='rsi_14'
                    ), row=3, col=1)  


# Update the layout by changing the figure size, hiding the legend and rangeslider
fig.update_layout(height=600,# width=1000, 
                  showlegend=True, 
                  xaxis_rangeslider_visible=False)

fig.show()